Part 4 

In [156]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import tensorflow as tf
import cv2
from sklearn import preprocessing


sns.set()

In [157]:
def initialize_parameters(n_x, n_h, n_y):
    
    np.random.seed(2) 
    
    W1 = np.random.randn(n_h, n_x) * 0.01
    b1 = np.zeros(shape=(n_h, 1))
    W2 = np.random.randn(n_y, n_h) * 0.01
    b2 = np.zeros(shape=(n_y, 1))
    
    assert (W1.shape == (n_h, n_x))
    assert (b1.shape == (n_h, 1))
    assert (W2.shape == (n_y, n_h))
    assert (b2.shape == (n_y, 1))
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [158]:
def relu(x):
    return x*(x>0)
def sigmoid(x):
    return 1/(1+np.exp(-x))
def tanh(x):
    return (np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))

In [159]:
def forward_pass(X, parameters):
    
    # to make forward pass calculations we need W1 and W2 so we will extract them from dictionary parameters
    W1 = parameters['W1']
    W2 = parameters['W2']
    b1 = parameters['b1']
    b2 = parameters['b2']

    #print(W1)
    # first layer calculations - hidden layer calculations
    Z1 = np.dot(W1, X) + b1
    A1 = tanh(Z1)  # activation in the first layer is tanh
    
    # output layer calculations
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2)# A2 are predictions, y_hat
    
    # cache values for backpropagation calculations
    cache = {'Z1':Z1,
             'A1':A1,
             'Z2':Z2,
             'A2':A2
            }
    # print(A2.shape)
    return A2, cache

In [160]:
def backward_pass(parameters, cache, X, Y):
    
    # unpack paramaeters and cache to get values for backpropagation calculations
    W1 = parameters['W1']
    W2 = parameters['W2']
    
    Z1 = cache['Z1']
    A1 = cache['A1']
    Z2 = cache['Z2']
    A2 = cache['A2']
    
    m = X.shape[1] # number of examples in a training set
    #print(A2)
    #print(A2.shape)
    dZ2= A2 - Y
    
    
    dW2 = (1 / m) * np.dot(dZ2, A1.T)
    db2 = (1 / m) * np.sum(dZ2, axis=1, keepdims=True)   # keepdims - prevents python to output rank 1 array (n,)
    
    dZ1 = np.multiply(np.dot(W2.T, dZ2), 1 - np.power(A1, 2)) # we use tanh activation function
    dW1 = (1 / m) * np.dot(dZ1, X.T)  
    db1 = (1 / m) * np.sum(dZ1, axis=1, keepdims=True)
    
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    
    return grads

In [161]:
def update_parameters(parameters, learning_rate, grads):
    
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    
    dW1 = grads['dW1']
    db1 = grads['db1']
    dW2 = grads['dW2']
    db2 = grads['db2']
    
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1
    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2
    
    # updated parameters
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [162]:
def NN_model(X,Y,n_h, num_iterations, learning_rate):
    
    n_x = X.shape[0] # size of an input layer = number of features 
    n_y = Y.shape[0] # size of an output layer
    parameters = initialize_parameters(n_x, n_h, n_y)
    
    #unpack parameters
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    
    for i in range(num_iterations):
        A2, cache = forward_pass(X, parameters)
        grads = backward_pass(parameters, cache, X, Y)
        parameters = update_parameters(parameters, learning_rate, grads)
        
    return parameters

In [163]:
def predict(parameters, X):
    
    A2, cache = forward_pass(X, parameters)
    predictions = np.round(A2)
    
    return predictions

In [164]:
 (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [165]:
labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [166]:
arr = [y_train.tolist()]
y_train = np.array(arr)

arr1 = [y_test.tolist()]
y_test = np.array(arr1)

In [167]:
x_train = np.array([cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in x_train])
x_test = np.array([cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in x_test])
#x_train.shape

In [178]:
x_train.size

51200000

In [71]:
# x_train  = x_train/255
# x_test  = x_test/255

In [83]:
# one_hot_encoder = preprocessing.OneHotEncoder(sparse=False)
# one_hot_encoder.fit(y_train)
# y_train = one_hot_encoder.transform(y_train)
# y_test = one_hot_encoder.transform(y_test)

In [84]:
# x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
# x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)

In [169]:
num_iterations = 20000
learning_rate = 0.1
n_h = 4
parameters_final = NN_model(x_train.T,y_train.T,n_h, num_iterations, learning_rate)

[[-4.16757847e-03 -5.62668272e-04 -2.13619610e-02  1.64027081e-02
  -1.79343559e-02 -8.41747366e-03  5.02881417e-03 -1.24528809e-02
  -1.05795222e-02 -9.09007615e-03  5.51454045e-03  2.29220801e-02
   4.15393930e-04 -1.11792545e-02  5.39058321e-03 -5.96159700e-03
  -1.91304965e-04  1.17500122e-02 -7.47870949e-03  9.02525097e-05
  -8.78107893e-03 -1.56434170e-03  2.56570452e-03 -9.88779049e-03
  -3.38821966e-03 -2.36184031e-03 -6.37655012e-03 -1.18761229e-02
  -1.42121723e-02 -1.53495196e-03 -2.69056960e-03  2.23136679e-02]
 [-2.43476758e-02  1.12726505e-03  3.70444537e-03  1.35963386e-02
   5.01857207e-03 -8.44213704e-03  9.76147160e-08  5.42352572e-03
  -3.13508197e-03  7.71011738e-03 -1.86809065e-02  1.73118467e-02
   1.46767801e-02 -3.35677339e-03  6.11340780e-03  4.79705919e-04
  -8.29135289e-03  8.77102184e-04  1.00036589e-02 -3.81092518e-03
  -3.75669423e-03 -7.44707629e-04  4.33496330e-03  1.27837923e-02
  -6.34679305e-03  5.08396243e-03  2.16116006e-03 -1.85861239e-02
  -4.1931

ValueError: ignored

In [117]:
y_train.shape

(50000, 1)